# Part 2: Time Series Modeling

In this notebook, you will implement functions to extract features from time series data and build ARIMA models.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from pathlib import Path
import os

# Set style for plots
plt.style.use('seaborn')
%matplotlib inline

## 1. Feature Extraction

Implement the `extract_time_series_features` function to calculate rolling window features.

In [6]:
def extract_time_series_features(data, window_size=60):
    """Extract rolling window features from time series data.
    
    Parameters
    ----------
    data : pd.DataFrame
        Preprocessed physiological data
    window_size : int
        Size of the rolling window in seconds
        
    Returns
    -------
    pd.DataFrame
        DataFrame containing extracted features for each signal
    """
    # Your code here
    # 1. Calculate rolling window statistics
    # 2. Include mean, std, min, max, and autocorrelation
    
    data = data.copy()
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.set_index('timestamp').sort_index()  
    
    features = []

    def autocorr(x, lag=1):
        """Autocorrelation at lag 1."""
        if len(x) <= lag:
            return 0
        return x.autocorr(lag=lag)

    signals = ['heart_rate', 'eda', 'temperature']
    
    for signal in signals:
        if signal not in data.columns:
            continue

        rolling = data[signal].rolling(f'{window_size}s')

        stats_df = pd.DataFrame({
            f'{signal}_mean': rolling.mean(),
            f'{signal}_std': rolling.std(),
            f'{signal}_min': rolling.min(),
            f'{signal}_max': rolling.max(),
            f'{signal}_autocorr_lag1': rolling.apply(lambda x: autocorr(x, lag=1), raw=False)
        })

        features.append(stats_df)

    # Combine features from each signal
    feature_df = pd.concat(features, axis=1)
    feature_df = feature_df.dropna().reset_index()

    return feature_df

    pass

In [ ]:
df_clean = pd.read_csv('data/processed/S1_processed.csv')
features_df = extract_time_series_features(df_clean, window_size=60)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)


,timestamp,heart_rate_mean,heart_rate_std,heart_rate_min,heart_rate_max,heart_rate_autocorr_lag1,eda_mean,eda_std,eda_min,eda_max,eda_autocorr_lag1,temperature_mean,temperature_std,temperature_min,temperature_max,temperature_autocorr_lag1
0,2018-10-13 12:56:11,106.127744,0.000000,106.127744,106.127744,1.000000e+00,0.021036,0.004982,0.010892,0.023386,-0.047845,22.510000,0.012649,22.49,22.53,-7.905694e-01
1,2018-10-13 12:56:12,106.127744,0.000000,106.127744,106.127744,1.000000e+00,0.021280,0.004594,0.010892,0.023386,-0.113595,22.512857,0.013801,22.49,22.53,-6.220800e-14
2,2018-10-13 12:56:13,106.127744,0.000000,106.127744,106.127744,1.000000e+00,0.021584,0.004338,0.010892,0.023706,-0.003646,22.515000,0.014142,22.49,22.53,2.282177e-01
3,2018-10-13 12:56:16,104.116131,6.671766,84.000000,106.127744,4.281459e-16,0.022075,0.003734,0.010892,0.023706,0.053735,22.511818,0.014013,22.49,22.53,1.904762e-01
4,2018-10-13 12:56:17,102.523120,8.421287,84.000000,106.127744,6.516530e-01,0.022158,0.003572,0.010892,0.023706,0.040152,22.510000,0.014771,22.49,22.53,3.685139e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44694,2018-12-05 22:58:49,130.779833,3.182419,123.150000,134.550000,9.709228e-01,0.026193,0.000522,0.024666,0.027869,-0.086938,24.897333,1.176879,24.21,30.11,7.723194e-01
44695,2018-12-05 22:58:50,130.636167,3.346579,122.730000,134.550000,9.739071e-01,0.026203,0.000530,0.024666,0.027869,-0.065847,24.937000,1.196689,24.21,30.11,7.801478e-01
44696,2018-12-05 22:58:51,130.480667,3.507771,122.400000,134.550000,9.765829e-01,0.026214,0.000525,0.024666,0.027869,-0.072939,24.976333,1.214385,24.21,30.11,7.868032e-01
44697,2018-12-05 22:58:52,130.307833,3.667447,121.950000,134.550000,9.792020e-01,0.027340,0.008705,0.024666,0.093525,0.006930,25.015667,1.230550,24.21,30.11,7.925546e-01


## 2. ARIMA Modeling

Implement the `build_arima_model` function to fit ARIMA models and generate diagnostic plots.

In [11]:
def build_arima_model(series, order=(1,1,1), output_dir='plots'):
    """Fit an ARIMA model to the time series and generate diagnostic plots.
    
    Parameters
    ----------
    series : pd.Series
        Time series data to model
    order : tuple
        (p,d,q) order of the ARIMA model
    output_dir : str
        Directory to save diagnostic plots
        
    Returns
    -------
    statsmodels.tsa.arima.model.ARIMAResults
        Fitted ARIMA model
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Your code here
    # 1. Fit ARIMA model
    # 2. Generate diagnostic plots:
    #    - Model fit plot
    #    - Residuals plot
    #    - Forecast plot
    # 3. Save plots to output directory

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Use signal name from the series if available
    signal_name = series.name if series.name else 'signal'
    base_name = f"{signal_name}_arima"

    # Fit ARIMA model
    model = ARIMA(series, order=order)
    model_fit = model.fit()

    # Plot observed vs fitted
    plt.figure(figsize=(10, 4))
    plt.plot(series.index, series, label='Observed', color='blue')
    plt.plot(series.index, model_fit.fittedvalues, label='Fitted', color='red')
    plt.title(f'ARIMA Fit: {signal_name}')
    plt.xlabel("Time")
    plt.ylabel(signal_name)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_fit.png"))
    plt.close()

    # Plot residuals
    residuals = model_fit.resid
    plt.figure(figsize=(10, 4))
    plt.plot(residuals, label='Residuals', color='green')
    plt.axhline(0, color='black', linestyle='--')
    plt.title(f'ARIMA Residuals: {signal_name}')
    plt.xlabel("Time")
    plt.ylabel("Residuals")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_residuals.png"))
    plt.close()

    return model_fit

    pass

In [13]:
series = df_clean['heart_rate']
fitted_model = build_arima_model(series, order=(1,1,1), output_dir='plots')
fitted_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:             heart_rate   No. Observations:                45713
Model:                 ARIMA(1, 1, 1)   Log Likelihood              -75680.337
Date:                Wed, 30 Apr 2025   AIC                         151366.674
Time:                        00:28:36   BIC                         151392.864
Sample:                             0   HQIC                        151374.912
                              - 45713                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9368      0.003    356.887      0.000       0.932       0.942
ma.L1         -0.8726      0.003   -300.446      0.000      -0.878      -0.867
sigma2         1.6053      0.000   4408.038      0.000       1.605       1.606
===================================================================================
Ljung-Box (L1) (Q):                  71.78   Jarque-Bera (JB):       17284091730.06
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               7.79   Skew:                            -1.50
Prob(H) (two-sided):                  0.00   Kurtosis:                      3015.41
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""